In [ ]:
# Library installer
!pip install googletrans
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15735 sha256=c8a6b90435ac6e145a173e24b3d259447fedce79566b70b609c3660bd5a20f32
  Stored in directory: /root/.cache/pip/wheels/27/f3/32/d4859d40071f07a5df0ab6fdc0076e78a8a786625dde2b4b2f
Successfully built googletrans
  

In [ ]:
# Loading Libraries
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from googletrans import Translator
from langdetect import detect_langs
import re
from sklearn.preprocessing import LabelEncoder
import numpy as np
from geopy.geocoders import Nominatim


In [ ]:
# Constants
df_gemi = pd.read_csv('/content/elevatedetailed_v2.csv')
df_elevate = pd.read_excel('/content/gemi.xlsx')

# Data Preprocessing

In [ ]:
# Merging the data files
df_elevate_gemi = pd.merge(df_gemi, df_elevate, on='id', how='left')

In [ ]:
# Replace blank space with _ 
# put your dataframes into a list

for col in df_elevate_gemi.columns:
  # check if the column name contains a space
  if ' ' in col:

    # rename the column by replacing spaces with underscores
    df_elevate_gemi.rename(columns={col: col.replace(' ', '_')}, inplace=True)

In [ ]:
df_elevate_gemi.head(2)

,Unnamed:_0.1,Unnamed:_0_x,id,Corporate_Name_x,Region,INDUSTRIES,TECHNOLOGY,BUSINESS_MODEL,REVENUE_MODEL,FUNDING_SOURCES,...,Unnamed:_0_y,Corporate_Name_y,Commercial_Titles,Date_Started,Legal_Type,Address,Website,kind_of_shares,no_of_shares,value_of_share
0,0,0,1,[i2.d] technologies,"Halandri, Greece",['Enterprise Software'],['Cloud Computing'],"['B2B', 'B2B2C']","['Commission Fee', 'SaaS', 'Subscription']",['VC'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,10,A.D.W.K. ADWORK,"ZOGRAFOU , Greece",['Advertising & Marketing (AdTech)'],"['AI', 'AR/VR', 'Data Analytics - Big Data', '...","['B2B', 'B2B2C']","['Price per Item', 'SaaS']",['Self'],...,0.0,A.D.W.K. ADWORK ΙΔΙΩΤΙΚΗ ΚΕΦΑΛΑΙΟΥΧΙΚΗ ΕΤΑΙΡΕΙΑ,ADWORK PC (Latin characters interpretation)ADW...,19/11/2021,PC,"XIROGIANNI 37, ATHINA, 15771",NaN,Capital,"1,600",€10.00





# Handling List




In [ ]:
def clean_and_encode(df, col_name, suffix):
    unique_values = df[col_name].unique()
    print(unique_values)
    null_values = df[col_name].isnull()
    print(null_values.sum())
    df[col_name + '_updated'] = df[col_name].replace('[]', '''['Other']''')
    unique_values_update = df[col_name + '_updated'].unique()
    print(unique_values_update)
    # convert the data to a list of lists
    df_list = df[col_name + '_updated'].apply(lambda x: eval(x)).tolist()
    y = [ [col_name +'_'+ elem for elem in inner_list] for inner_list in df_list]
    print("********************** Printing DF LIST*************", y)
    # use MultiLabelBinarizer to perform one-hot encoding
    mlb = MultiLabelBinarizer()
    encoded_data = mlb.fit_transform(df_list)
    # create a new dataframe with the encoded data
    encoded_df = pd.DataFrame(encoded_data, columns=[ suffix + '_' + mlb.classes_])
    print(encoded_df)
    return encoded_df

In [ ]:
# Passing the volumn values to the function

df_encoded_bm = clean_and_encode(df_elevate_gemi, 'BUSINESS_MODEL', 'BM')
df_encoded_fs = clean_and_encode(df_elevate_gemi, 'FUNDING_SOURCES', 'FS')
df_encoded_rm = clean_and_encode(df_elevate_gemi, 'REVENUE_MODEL', 'RM')
df_encoded_in = clean_and_encode(df_elevate_gemi, 'INDUSTRIES', 'INDUS')
df_encoded_th = clean_and_encode(df_elevate_gemi, 'TECHNOLOGY', 'TECH')

In [ ]:
df_encoded = pd.concat([df_encoded_bm, df_encoded_fs, df_encoded_rm, df_encoded_in, df_encoded_th], axis=1)

# drop the original columns
#df_elevate_gemi = df_elevate_gemi.drop(['BUSINESS_MODEL', 'FUNDING_SOURCES', 'REVENUE_MODEL', 'INDUSTRIES', 'TECHNOLOGY'], axis=1)

# merge the encoded dataframe with the original dataframe
df_elevate_gemi = pd.concat([df_elevate_gemi, df_encoded], axis=1)


In [ ]:
df_elevate_gemi.head(2)

,Unnamed:_0.1,Unnamed:_0_x,id,Corporate_Name_x,Region,INDUSTRIES,TECHNOLOGY,BUSINESS_MODEL,REVENUE_MODEL,FUNDING_SOURCES,...,Hardware,IoT,Nanotech,Networks,Other,Quantum Computing,Robotics,Sensors,Software,Web or Mobile Application
0,0,0,1,[i2.d] technologies,halandri,['Enterprise Software'],['Cloud Computing'],"['B2B', 'B2B2C']","['Commission Fee', 'SaaS', 'Subscription']",['VC'],...,0,0,0,0,0,0,0,0,0,0
1,1,1,10,A.D.W.K. ADWORK,zografou,['Advertising & Marketing (AdTech)'],"['AI', 'AR/VR', 'Data Analytics - Big Data', '...","['B2B', 'B2B2C']","['Price per Item', 'SaaS']",['Self'],...,0,0,0,0,0,0,0,0,1,1


# Handling Dictionary

In [ ]:
# replace null or empty values with default value
df_elevate_gemi['FUNDING'].fillna("[{'title': 'None', 'amount': 'None', 'year': '0', 'type': 'None'}]", inplace=True)

In [ ]:
def extract_data(row):
    if isinstance(row, str):
        row = eval(row)  # convert string to list of dictionaries
    elif np.isnan(row):
        return pd.Series({'title': np.nan, 'amount': np.nan, 'year': np.nan, 'type': np.nan})
    latest_year = max(row, key=lambda x: int(x['year']))['year']
    latest_dict = [d for d in row if d['year'] == latest_year][0]
    # split title and type strings into lists, ignoring commas inside brackets and single quotes
    # title_list = re.findall(r"[\w\s-]+[\w\s−]+[\w\s−]+", latest_dict['title'])
    title_list = re.findall(r"[^',]+(?:[^',]+)*", latest_dict['title'])
    type_list = re.findall(r"[^',]+(?:[^',]+)*", latest_dict['type'])
    return pd.Series({'title': title_list, 'amount': latest_dict['amount'], 'year': latest_year, 'type': type_list})

In [ ]:
# apply the function to the FUNDING column
df_elevate_gemi[['title', 'amount', 'year', 'type']] = df_elevate_gemi['FUNDING'].apply(extract_data)

# print the updated dataframe
df_elevate_gemi.head(6)

In [ ]:
# dictionary like preposcessing for similar columns

df_elevate_gemi['PATENTS_COUNT'] = df_elevate_gemi['PATENTS'].apply(lambda x: 0 if pd.isna(x) else x.count("'title'"))

df_elevate_gemi['AWARDS_COUNT'] = df_elevate_gemi['AWARDS'].apply(lambda x: 0 if pd.isna(x) else x.count("'title'"))

df_elevate_gemi['INCUBATIONS/ACCELERATIONS_COUNT'] = df_elevate_gemi['INCUBATIONS/ACCELERATIONS'].apply(lambda x: 0 if pd.isna(x) else x.count("'title'"))

# Label encoding

In [ ]:
# create an instance of the LabelEncoder class
encoder = LabelEncoder()

# encode the 'Funding_State' column
df_elevate_gemi['Funding_State'] = encoder.fit_transform(df_elevate_gemi['Funding_State'])

# print the resulting dataframe with the encoded 'Funding_State' column
print(df_elevate_gemi)

In [ ]:
value = df_elevate_gemi.loc[df_elevate_gemi['id'] == 101, 'Funding_State'].values[0]
print(value)

0


# Handling Gemi Preprocessing

In [ ]:
# convert the columns to string type

df_elevate_gemi['value_of_share'] = df_elevate_gemi['value_of_share'].astype(str)
df_elevate_gemi['no_of_shares'] = df_elevate_gemi['no_of_shares'].astype(str)

# replace the euro symbol with a space in the 'value_of_share' column

df_elevate_gemi['value_of_share'] = df_elevate_gemi['value_of_share'].str.replace('€', '')
df_elevate_gemi['value_of_share'] = df_elevate_gemi['value_of_share'].str.replace(',', '')

# replace 'Ordinary' with 1 in the 'no_of_shares' column

df_elevate_gemi['no_of_shares'] = df_elevate_gemi['no_of_shares'].str.replace('Ordinary', '1')
df_elevate_gemi['no_of_shares'] = df_elevate_gemi['no_of_shares'].str.replace(',', '')

df_elevate_gemi['Date_Started_new'] = pd.to_datetime(df_elevate_gemi['Date_Started'],  format='%d/%m/%Y')

# Replace 'nan' strings with np.nan

df_elevate_gemi.replace('nan', np.nan, inplace=True)

# Convert the columns to numeric types

df_elevate_gemi['value_of_share'] = pd.to_numeric(df_elevate_gemi['value_of_share'])
df_elevate_gemi['no_of_shares'] = pd.to_numeric(df_elevate_gemi['no_of_shares'])

# Perform computations on the new dataframe

df_elevate_gemi['capital'] = df_elevate_gemi['value_of_share'] * df_elevate_gemi['no_of_shares']

# create a dictionary to map values to codes

legal_type_dict = {'Energy Community': 0, 'LLC':1, 'PC': 2, 'SA': 3, 'SOLE':4}
kind_of_shares_dict = {'Capital':0, 'Anonymous':0, 'non-Capital':1, 'Guarantee':2}

# use the dictionary to assign codes to the values in the Legal_Type column

df_elevate_gemi['Legal_Type_Code'] = df_elevate_gemi['Legal_Type'].map(legal_type_dict)
df_elevate_gemi['kind_of_shares_Code'] = df_elevate_gemi['kind_of_shares'].map(kind_of_shares_dict)

# REGION

In [ ]:
# Code to lower and strip regions from 'Region' Column
df_elevate_gemi["Region"] = (df_elevate_gemi["Region"].str.lower()).str.replace(r', greece', '')

 # Code to replace all spelling errors in column values
value_corrections = {'ag paraskeyh': 'aghia paraskevi',
                    'aghia paraskevi, attikis': 'aghia paraskevi',
                    'aliartos viotia' : 'aliartos',
                    'analispi hersonisos': 'analispi',
                    'axarnes': 'acharnes',
                    'likovrisi - marousi ': 'lykovrysi',
                    'lycovrysi': 'lykovrysi',
                    'n. psyhiko athina': 'athina',
                    'neo psychico': 'neo',
                    'pilea': 'pylaia',
                    'pilea - thessaloniki': 'pylaia',
                    'pireas': 'piraeus',
                    'thesssaloniki': 'thessaloniki',
                    'thissions, athens': 'athens'}

df_elevate_gemi['Region'] = df_elevate_gemi['Region'].replace(value_corrections)

In [ ]:
# Function to call api to extract region name
def get_greece_region_name(input_str):
  '''Function returns region name'''
  geolocator = Nominatim(user_agent="my_geocoder")
  location = geolocator.geocode(input_str + ", Greece", language="en")

  if location is not None:
    if 'Regional Unit' in location.raw['display_name'].split(',')[-3]:
      return location.raw['display_name'].split(',')[-2]
    else:
      return location.raw['display_name'].split(',')[-3]
  else:
      return input_str

In [ ]:
# Code to replace all Region values
df_elevate_gemi["New_Region"] = df_elevate_gemi["Region"].apply(get_greece_region_name)

In [ ]:
df_elevate_gemi.to_excel("Cleaned_data.xlsx", index=False)

# Dropping Unused Columns

In [ ]:
for col in df_elevate_gemi.columns:
  print(col)

Unnamed:_0.1
Unnamed:_0_x
id
Corporate_Name_x
Region
INDUSTRIES
TECHNOLOGY
BUSINESS_MODEL
REVENUE_MODEL
FUNDING_SOURCES
ONE_LINER
About
Solution_&_Product
Innovation_&_Scalability
Traction_&_achievements
Team_&_Why_Us
Funding_Needs
FUNDING
PATENTS
AWARDS
INCUBATIONS/ACCELERATIONS
Funding_State
region_population
Unnamed:_0_y
Corporate_Name_y
Commercial_Titles
Date_Started
Legal_Type
Address
Website
kind_of_shares
no_of_shares
value_of_share
BUSINESS_MODEL_updated
FUNDING_SOURCES_updated
REVENUE_MODEL_updated
INDUSTRIES_updated
TECHNOLOGY_updated
B2B
B2B2C
B2C
B2G
C2C
Other
Angel
Crowd
Grant
Loan
Other
Self
VC
Advertising
Commission Fee
Licencing
Marketplace/Brokerage
Other
Price per Item
Price per Use
SaaS
Subscription
Advertising & Marketing (AdTech)
AgriTech / FoodTech
Art, Cultural & Creative Industries
Data Analytics - Big Data
Defence Systems
EduTech - Education
Enterprise Software
Entertainment/Media (Games, Sports, Social)
Environment & Energy (GreenTech, CleanTech)
FinTech – Fin

In [ ]:
def drop_columns(df, columns_to_drop):
    return df.drop(columns_to_drop, axis=1)

In [ ]:
columns_to_drop = ['Unnamed:_0.1','Unnamed:_0_x','ONE_LINER','About','Solution_&_Product','Innovation_&_Scalability',
'Traction_&_achievements','Team_&_Why_Us','Funding_Needs','Unnamed:_0_y','Corporate_Name_y','Commercial_Titles','Address',
'Website']
df_elevate_gemi = drop_columns(df_elevate_gemi, columns_to_drop)
print(df_elevate_gemi)
